# Steps

1. Build a `RunOptions` generator with the parameters of interest, namely $dim_S, J_S, \delta_S, U_S, dim_A, J_A, \delta_A, U_A, \alpha_{xx}, \alpha_{xz}, \alpha_{zx}, \alpha_{zz}, t$. We need a decent range for $\delta_S$ for the Bayesian update side of things. We assume a uniform prior on the cartesian product.
2. Calculate the probabilities for the $J_z$ measurement for each value. This is a `dict`, as $J_z$ can have multiple outcomes.
3. Use the probabilities to build the expected likelihood function over all inputs, for a given range of $N_{trials}$. This also requires a "True" set of values for each parameter.
4. Use the likelihood to build the marginal distribution for all variables.
5. Save the std variation for each estimated variable!
6. Plot $\Delta$ agains the $N_{trials}$, keeping the parameters of the plot saved

In [1]:
from dataclasses import dataclass, field
from src.angular_momentum import generate_spin_matrices
from tqdm import tqdm
import functools
import itertools
import numpy as np
import pandas as pd
import scipy

# Create new `pandas` methods which use `tqdm` progress
# (can use tqdm_gui, optional kwargs, etc.)
tqdm.pandas()

In [2]:
@dataclass
class Settings:
    dim_s: int
    dim_a: int
    probability_error_tolerance: float
    system_jx: np.array = field(init=False)
    system_jz: np.array = field(init=False)
    ancilla_jx: np.array = field(init=False)
    ancilla_jz: np.array = field(init=False)
    initial_state: np.array = field(init=False)

    def __post_init__(self):
        self.system_jx, self.system_jz = generate_spin_matrices(dim=self.dim_s)
        self.ancilla_jx, self.ancilla_jz = generate_spin_matrices(dim=self.dim_a)
        self.initial_state = np.zeros(self.dim_s * self.dim_a)
        self.initial_state[0] = 1
        self.initial_state = np.outer(self.initial_state, self.initial_state)

    def generate_hamiltonian(
        self,
        j_s: float,
        u_s: float,
        delta_s: float,
        j_a: float,
        u_a: float,
        delta_a: float,
        alpha_xx: float,
        alpha_xz: float,
        alpha_zx: float,
        alpha_zz: float,
    ) -> np.array:
        system_hamiltonian = np.kron(
            -1 * j_s * self.system_jx + u_s * self.system_jz @ self.system_jz + delta_s * self.system_jz,
            np.divide(np.eye(self.dim_a), self.dim_a)
        )
        ancillary_hamiltonian = np.kron(
            np.divide(np.eye(self.dim_s), self.dim_s),
            -1 * j_a * self.ancilla_jx + u_a * self.ancilla_jz @ self.ancilla_jz + delta_a * self.ancilla_jz
        )
        interaction_hamiltonian = functools.reduce(
            lambda x, y: x + y,
            [
                alpha_xx * np.kron(self.system_jx, self.ancilla_jx),
                alpha_xz * np.kron(self.system_jx, self.ancilla_jz),
                alpha_zx * np.kron(self.system_jz, self.ancilla_jx),
                alpha_zz * np.kron(self.system_jz, self.ancilla_jz),
            ]
        )
        return system_hamiltonian + ancillary_hamiltonian + interaction_hamiltonian

    def trace_out_ancillary(self, state: np.array):
        return np.trace(
            np.array(state).reshape(self.dim_s, self.dim_a, self.dim_s, self.dim_a),
            axis1=1,
            axis2=3
        )

    @staticmethod
    def calculate_final_state(
        hamiltonian: np.array,
        initial_state: np.ndarray,
        t: float = 0,
    ) -> np.array:
        return scipy.linalg.expm(-1j * t * hamiltonian) @ initial_state @ scipy.linalg.expm(1j * t * hamiltonian)

    def calculate_probabilities(self, final_state: np.array) -> np.array:
        system_state = self.trace_out_ancillary(state=final_state)
        probabilities = [np.abs(x)**2 for x in np.diag(system_state)] # TODO: Ensure these actually up to 1!!!
        # assert np.abs(np.sum(probabilities) - 1) < settings.probability_error_tolerance, f"⚠ The observed probabilities {probabilities} are unphysical by {np.abs(np.sum(probabilities) - 1):.5f}%"
        return probabilities

# Initial state $\ket{0} \otimes \ket{0}$

In [3]:
settings = Settings(
    dim_s= 2,
    dim_a= 1,
    probability_error_tolerance= .001,
)

In [4]:
pd.DataFrame(settings.initial_state).style.background_gradient(cmap='viridis')

,0,1
0,1.000000,0.000000
1,0.000000,0.000000


# Create generator object

In [5]:
true_values = {
    "j_s": .2,
    "u_s": .1,
    "delta_s": 1.2,
    "j_a": .3,
    "u_a": .1,
    "delta_a": 1,
    "alpha_xx": 0,
    "alpha_xz": 0,
    "alpha_zx": 0,
    "alpha_zz": 0,
    "time": 5,
}
df = pd.DataFrame([
    {
        "j_s": j_s,
        "u_s": u_s,
        "delta_s": delta_s,
        "j_a": j_a,
        "u_a": u_a,
        "delta_a": delta_a,
        "alpha_xx": alpha_xx,
        "alpha_xz": alpha_xz,
        "alpha_zx": alpha_zx,
        "alpha_zz": alpha_zz,
        "time": time
    }
    for j_s, u_s, delta_s, j_a, u_a, delta_a, alpha_xx, alpha_xz, alpha_zx, alpha_zz, time
    in itertools.product(
        np.linspace(.099, .101, 3), # j_s: float,
        np.linspace(.099, .101, 3), # u_s: float,
        np.linspace(1,2,101), # delta_s: float,
        [0], # j_a: float,
        [0], # u_a: float,
        [1], # delta_a: float,
        [0], # alpha_xx: float,
        [0], # alpha_xz: float,
        [0], # alpha_zx: float,
        [0], # alpha_zz: float,
        [3,5,10], # time: float
    )],
)
df.sample(10).style.background_gradient(cmap='viridis', axis=0)

,j_s,u_s,delta_s,j_a,u_a,delta_a,alpha_xx,alpha_xz,alpha_zx,alpha_zz,time
1011,0.100000,0.099000,1.340000,0,0,1,0,0,0,0,3
1802,0.100000,0.101000,1.950000,0,0,1,0,0,0,0,10
391,0.099000,0.100000,1.290000,0,0,1,0,0,0,0,5
1650,0.100000,0.101000,1.450000,0,0,1,0,0,0,0,3
1927,0.101000,0.099000,1.360000,0,0,1,0,0,0,0,5
1853,0.101000,0.099000,1.110000,0,0,1,0,0,0,0,10
2684,0.101000,0.101000,1.860000,0,0,1,0,0,0,0,10
844,0.099000,0.101000,1.790000,0,0,1,0,0,0,0,5
2261,0.101000,0.100000,1.460000,0,0,1,0,0,0,0,10
933,0.100000,0.099000,1.080000,0,0,1,0,0,0,0,3


# Calculate measurement probabilities

In [6]:
def calculate_final_probabilities(row: pd.Series) -> np.array:
    hamiltonian = settings.generate_hamiltonian(
        j_s = row["j_s"],
        u_s = row["u_s"],
        delta_s = row["delta_s"],
        j_a = row["j_a"],
        u_a = row["u_a"],
        delta_a = row["delta_a"],
        alpha_xx = row["alpha_xx"],
        alpha_xz = row["alpha_xz"],
        alpha_zx = row["alpha_zx"],
        alpha_zz = row["alpha_zz"]
    )
    final_state = settings.calculate_final_state(
        hamiltonian = hamiltonian,
        initial_state = settings.initial_state,
        t = row["time"],
    )
    # return final_state
    final_probabilities = settings.calculate_probabilities(final_state)
    return final_probabilities
    # return np.divide(final_probabilities, np.sum(final_probabilities))

df["probabilities"] = df.progress_apply(calculate_final_probabilities, axis=1) # These are the measurement probabilities, i.e. Prob[J_z=k] for k in range(0, dim_s)
df["final_state_error"] = df.progress_apply(lambda row: np.abs(np.sum(row["probabilities"]) - 1), axis=1)

df.sample(30).style.background_gradient(cmap='viridis', axis=0)

100%|██████████| 2727/2727 [00:00<00:00, 64828.02it/s]


,j_s,u_s,delta_s,j_a,u_a,delta_a,alpha_xx,alpha_xz,alpha_zx,alpha_zz,time,probabilities,final_state_error
1018,0.100000,0.099000,1.360000,0,0,1,0,0,0,0,5,"[0.9992482634427746, 1.4133008935868313e-07]",0.000752
624,0.099000,0.101000,1.060000,0,0,1,0,0,0,0,3,"[0.9827916259773714, 7.467595412474191e-05]",0.017134
1851,0.101000,0.099000,1.110000,0,0,1,0,0,0,0,3,"[0.983810539484678, 6.60604907950813e-05]",0.016123
1723,0.100000,0.101000,1.690000,0,0,1,0,0,0,0,5,"[0.9945174211244514, 7.53533852088276e-06]",0.005475
1335,0.100000,0.100000,1.410000,0,0,1,0,0,0,0,3,"[0.9927340398169298, 1.3246713259892846e-05]",0.007253
271,0.099000,0.099000,1.900000,0,0,1,0,0,0,0,5,"[0.9946025910422831, 7.302727099000306e-06]",0.005390
1228,0.100000,0.100000,1.050000,0,0,1,0,0,0,0,5,"[0.9958007657817969, 4.4176723041403165e-06]",0.004195
893,0.099000,0.101000,1.950000,0,0,1,0,0,0,0,10,"[0.9994354039359523, 7.971468377166341e-08]",0.000565
1745,0.100000,0.101000,1.760000,0,0,1,0,0,0,0,10,"[0.9978854896318378, 1.118971876105434e-06]",0.002113
916,0.100000,0.099000,1.020000,0,0,1,0,0,0,0,5,"[0.9943007917385523, 8.143465935015656e-06]",0.005691


# Calculate likelihoods

In [7]:
true_likelihoods = calculate_final_probabilities(true_values)
true_likelihoods /= np.prod(true_likelihoods)
true_likelihoods

array([1.36660704e+07, 1.00054123e+00])

In [8]:
df = df.merge(
    pd.DataFrame([{"n_trials": 2**x} for x in range(16)]),
    how='cross'
)
df.astype({'n_trials': 'int32'}, copy=False)
df.shape

(43632, 14)

In [9]:
np.power(np.subtract(np.linspace(1,3,11),1), np.linspace(0,2,11))

array([1.        , 0.72477966, 0.69314484, 0.73602192, 0.83651164,
       1.        , 1.24456475, 1.6016929 , 2.12125057, 2.8806501 ,
       4.        ])

In [10]:
np.array([1,2]) * np.array([3,4])

array([3, 8])

In [11]:
def calculate_log_likelihood(true_probabilities: np.array, expected_probabilities: np.array, n_trials: float) -> float:
    true_probabilities = np.array(true_probabilities)
    expected_probabilities = np.array(expected_probabilities)
    return np.sum([
        scipy.special.loggamma(n_trials * np.sum(expected_probabilities)),
        np.sum((n_trials * expected_probabilities - 1)*np.log(true_probabilities)),
        np.sum(scipy.special.loggamma(n_trials * expected_probabilities)),
    ])

df["log_likelihood"] = df.progress_apply(lambda row: calculate_log_likelihood(
    true_probabilities=true_likelihoods,
    expected_probabilities=row["probabilities"],
    n_trials=row["n_trials"]
), axis=1)
df["log_likelihood"] -= np.max(df["log_likelihood"])

100%|██████████| 43632/43632 [00:01<00:00, 25793.61it/s]


In [12]:
# df.sample(30).style.background_gradient(cmap='viridis', axis=0)
df.sort_values("log_likelihood", ascending=False).head(30).style.background_gradient(cmap='viridis', axis=0)

,j_s,u_s,delta_s,j_a,u_a,delta_a,alpha_xx,alpha_xz,alpha_zx,alpha_zz,time,probabilities,final_state_error,n_trials,log_likelihood
30319,0.101000,0.099000,1.250000,0,0,1,0,0,0,0,5,"[0.9999994672795721, 7.094778268043831e-14]",0.000001,32768,0.000000
35167,0.101000,0.100000,1.250000,0,0,1,0,0,0,0,5,"[0.999999467279571, 7.094778268043255e-14]",0.000001,32768,-0.000000
40015,0.101000,0.101000,1.250000,0,0,1,0,0,0,0,5,"[0.9999994672795695, 7.094778268044107e-14]",0.000001,32768,-0.000000
20623,0.100000,0.100000,1.250000,0,0,1,0,0,0,0,5,"[0.9999994445407147, 7.713377610532992e-14]",0.000001,32768,-0.111334
15775,0.100000,0.099000,1.250000,0,0,1,0,0,0,0,5,"[0.9999994445407138, 7.713377610532815e-14]",0.000001,32768,-0.111334
25471,0.100000,0.101000,1.250000,0,0,1,0,0,0,0,5,"[0.9999994445407129, 7.713377610532433e-14]",0.000001,32768,-0.111334
6079,0.099000,0.100000,1.250000,0,0,1,0,0,0,0,5,"[0.9999994223480445, 8.342046996717747e-14]",0.000001,32768,-0.216756
1231,0.099000,0.099000,1.250000,0,0,1,0,0,0,0,5,"[0.9999994223480434, 8.342046996718256e-14]",0.000001,32768,-0.216756
10927,0.099000,0.101000,1.250000,0,0,1,0,0,0,0,5,"[0.9999994223480412, 8.342046996718595e-14]",0.000001,32768,-0.216756
38207,0.101000,0.100000,1.880000,0,0,1,0,0,0,0,10,"[0.9999992751100003, 1.3136642829538273e-13]",0.000001,32768,-0.850452


# Take likelihood marginals

In [13]:
true_likelihoods

array([1.36660704e+07, 1.00054123e+00])

In [18]:
np.exp([1,2,3])

array([ 2.71828183,  7.3890561 , 20.08553692])

In [19]:
# for parameter in true_values.keys():
#     temp_df = df[[parameter, "n_trials", "likelihood"]]

parameter = "delta_s"
temp_df = df[[parameter, "n_trials", "log_likelihood"]]
temp_df.groupby(["n_trials", "delta_s"])["log_likelihood"].agg(lambda row: np.sum(np.exp(row)))

n_trials  delta_s
1         1.00        0.000000e+00
          1.01        0.000000e+00
          1.02        0.000000e+00
          1.03        0.000000e+00
          1.04        0.000000e+00
                         ...      
32768     1.96       2.955513e-109
          1.97        3.352855e-93
          1.98        1.493572e-78
          1.99        2.813945e-65
          2.00        2.423271e-53
Name: log_likelihood, Length: 1616, dtype: float64

In [ ]:
temp_df = df[[parameter, "n_trials", "likelihood"]]
temp_df.sample(10).style.background_gradient(cmap='viridis', axis=0)

In [ ]:
temp_df.groupby("n_trials")["likelihood"].sum()